# Homework 1: Montecarlo and TD methods

Let's implement these methods for a simple toy case with discrete and action spaces.

- Implement First-visit Montecarlo for Frozen lake (1p)
- Implement Sarsa and Q-Learning for (1p each)
- How does the value function V look for the different Q functions you found above? You can look at tutorial 3 for ideas how to do this plot. (1p) 

The code below can help you get started with TD. For inspiration about Montecarlo, look at tutorial 3.


In [1]:
import numpy as np
import gym

In [2]:
env = gym.make("FrozenLake-v0")

In [4]:
def epsilon_greedy_policy(Q, epsilon, actions):
    """ Q is a numpy array = qualities of (state, action) pairs, 
        epsilon between 0,1 
        actions = list of actions"""
    
    def policy_fn(state):
        if np.random.rand()>epsilon:
            action = np.argmax(Q[state,:])
        else:
            action = np.random.choice(actions)
        return action
    
    return policy_fn

In [5]:
env.observation_space.n


16

In [6]:
env.action_space.n

4

In [3]:
Q = np.zeros([env.observation_space.n, env.action_space.n])
actions = range(env.action_space.n)
score = [] 

gamma = 0.99 # discount factor
alpha = 0.1  # 
n_episodes = 100
 
for j in range(n_episodes):
    done = False
    state = env.reset()
    
    # Play randomly 10 episodes, then reduce slowly the randomness
    policy = epsilon_greedy_policy(Q, epsilon=10./(j+1), actions = actions ) 
    
    
    ### Generate sample episode
    t=0
    while not done:
        t+=1
        action = policy(state)    
        new_state, reward, done, _ =  env.step(action)
        new_action = policy(new_state)
        
        #Book-keeping
        if done:
            # YOUR CODE GOES HERE
            pass
        else:
            # YOUR CODE GOES HERE
            pass
            
        state, action = new_state, new_action
            
        if done:
            if len(score) < 100:
                score.append(reward)
            else:
                score[j % 100] = reward
                
                
            if (j+1)%1000 == 0:
                print("INFO: Episode {} finished after {} timesteps with r={}. \
                Running score: {}".format(j+1, t, reward, np.mean(score)))
            



[2018-03-06 08:21:11,952] Making new env: FrozenLake-v0


In [ ]:
env.close()